Check Python, Tensorflow, CUDA versions

In [ ]:
!python --version
!python3 -c 'import tensorflow as tf; print(tf.__version__)'
!nvcc -V

Python 3.7.10
2021-05-23 19:18:24.481383: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2.4.1
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


DATASET Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import a manually uploaded Schmugge subsplit-augmented(eg: dark) dataset
import_schmugge_aug = False

if import_schmugge_aug:
    !rm -rf dataset/Schmugge
    !unzip drive/MyDrive/schm/Schmugge.zip -d dataset # schmugge dark aug

Extract the dataset(s)

In [ ]:
!rm -rf dataset

# full body
!unzip drive/MyDrive/datasets/fullbody/ECU.zip -d dataset
#!unzip drive/MyDrive/datasets/fullbody/VDM.zip -d dataset
#!unzip drive/MyDrive/datasets/fullbody/Uchile.zip -d dataset
#!unzip drive/MyDrive/datasets/fullbody/Pratheepan.zip -d dataset

# hand
!unzip drive/MyDrive/datasets/hand/HGR_small.zip -d dataset

# face
!unzip drive/MyDrive/datasets/face/Schmugge.zip -d dataset

# abdomen
#!unzip drive/MyDrive/datasets/abdomen/abd-skin.zip -d dataset

# import-db.json files
!unzip drive/MyDrive/datasets/imports/dataset_imports.zip -d dataset

Dataset Organization Utils

In [ ]:
import os, re, sys, json
import cv2 # to load, save, process images
import imghdr # to check if a file is an image
import numpy as np
from random import shuffle
from math import floor

# remember that Pratheepan dataset has one file with comma in the filename
csv_sep = '?'


def get_training_and_testing_sets(file_list: list, split: float = 0.7):
    print(file_list)
    split_index = floor(len(file_list) * split)
    training = file_list[:split_index]
    testing = file_list[split_index:]
    return training, testing

# Get the variable part of a filename into a dataset
def get_variable_filename(filename: str, format: str) -> str:
    if format == '':
        return filename

    # re.fullmatch(r'^img(.*)$', 'imgED (1)').group(1)
    # re.fullmatch(r'^(.*)-m$', 'att-massu.jpg-m').group(1)
    match =  re.fullmatch('^{}$'.format(format), filename)
    if match:
        return match.group(1)
    else:
        #print('Cannot match {} with pattern {}'.format(filename, format))
        return None

# args eg: datasets/ECU/skin_masks datasets/ECU/original_images datasets/ecu/
# note: NonDefined, TRain, TEst, VAlidation
def analyze_dataset(gt: str, ori: str, root_dir: str, note: str = 'nd',
                    gt_filename_format: str = '', ori_filename_format: str = '',
                    gt_ext: str = '', ori_ext: str = '') -> None:
    out_analysis_filename = 'data.csv'

    out_analysis = os.path.join(root_dir, out_analysis_filename)
    analyze_content(gt, ori, out_analysis, note = note,
                    gt_filename_format = gt_filename_format,
                    ori_filename_format = ori_filename_format,
                    gt_ext = gt_ext, ori_ext = ori_ext)

# creates a file with lines like: origina_image1.jpg, skin_mask1.png, tr
def analyze_content(gt: str, ori: str, outfile: str, note: str = 'nd',
                    gt_filename_format: str = '', ori_filename_format: str = '',
                    gt_ext: str = '', ori_ext: str = '') -> None:
    # images found
    i = 0

    # append to data file
    with open(outfile, 'a') as out:

        for gt_file in os.listdir(gt):
            gt_path = os.path.join(gt, gt_file)

            # controlla se e' un'immagine (per evitare problemi con files come thumbs.db)
            if not os.path.isdir(gt_path) and imghdr.what(gt_path) != None:
                matched = False
                gt_name, gt_e = os.path.splitext(gt_file)
                gt_identifier = get_variable_filename(gt_name, gt_filename_format)

                if gt_identifier == None:
                    continue

                if gt_ext and gt_e != '.' + gt_ext:
                    continue
                
                for ori_file in os.listdir(ori):
                    ori_path = os.path.join(ori, ori_file)
                    ori_name, ori_e = os.path.splitext(ori_file)
                    ori_identifier = get_variable_filename(ori_name, ori_filename_format)
                    
                    if ori_identifier == None:
                        continue

                    if ori_ext and ori_e != '.' + ori_ext:
                        continue
                    
                    # try to find a match (original image - gt)
                    if gt_identifier == ori_identifier:
                        out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note}\n")
                        i += 1
                        matched = True
                        break
                
                if not matched:
                    print(f'No matches found for {gt_identifier}')
            else:
                print(f'File {gt_path} is not an image')
        
        print(f"Found {i} images")

# Does a simple processing on all dataset images based on a few operations.
# Used to make ground truth masks uniform across the datasets.
# 
# (JSON) "processpipe" : "png,skin=255_255_255,invert"
#        "processout" : "out/process/folder" 
# skin=... vuol dire che la regola per binarizzare è: tutto quello che non è skin va a nero, skin a bianco
# bg=... viceversa
#    (quindi skin e bg fanno anche binarizzazione!)
# *il processing viene fatto nell'ordine scritto!!!
def process_images(data_dir: str, process_pipeline: str, out_dir = '',
                   im_filename_format: str = '', im_ext: str = '') -> str:

    # loop mask files
    for im_basename in os.listdir(data_dir):
        im_path = os.path.join(data_dir, im_basename)
        im_filename, im_e = os.path.splitext(im_basename)

        # controlla se e' un'immagine (per evitare problemi con files come thumbs.db)
        if not os.path.isdir(im_path) and imghdr.what(im_path) != None:
            if out_dir == '':
                out_dir = os.path.join(data_dir, 'processed')

            os.makedirs(out_dir, exist_ok=True)

            im_identifier = get_variable_filename(im_filename, im_filename_format)
            if im_identifier == None:
                continue
            
            if im_ext and im_e != '.' + im_ext:
                continue

            # load image
            im = cv2.imread(im_path)

            # prepare path for out image
            im_path = os.path.join(out_dir, im_basename)

            for operation in process_pipeline.split(','):
                # binarize. Rule: what isn't skin is black
                if operation.startswith('skin'):
                    # inspired from https://stackoverflow.com/a/53989391
                    bgr_data = operation.split('=')[1]
                    bgr_chs = bgr_data.split('_')
                    b = int(bgr_chs[0])
                    g = int(bgr_chs[1])
                    r = int(bgr_chs[2])
                    lower_val = (b, g, r)
                    upper_val = lower_val
                    # Threshold the image to get only selected colors
                    # what isn't skin is black
                    mask = cv2.inRange(im, lower_val, upper_val)
                    im = mask
                # binarize. Rule: what isn't bg is white
                elif operation.startswith('bg'):
                    bgr_data = operation.split('=')[1]
                    bgr_chs = bgr_data.split('_')
                    b = int(bgr_chs[0])
                    g = int(bgr_chs[1])
                    r = int(bgr_chs[2])
                    lower_val = (b, g, r)
                    upper_val = lower_val
                    # Threshold the image to get only selected colors
                    mask = cv2.inRange(im, lower_val, upper_val)
                    #cv2_imshow(mask) #debug
                    # what isn't bg is white
                    sk = cv2.bitwise_not(mask)
                    im = sk
                # invert image
                elif operation == 'invert':
                    im = cv2.bitwise_not(im)
                # convert to png
                elif operation == 'png':
                    im_path = os.path.join(out_dir, im_filename + '.png')
                # reload image
                elif operation == 'reload':
                    im = cv2.imread(im_path)
                else:
                    print(f'Image processing operation unknown: {operation}')

            # save processing 
            cv2.imwrite(im_path, im)

    return out_dir

# update the csv by adding a split from a different-format file (1 column split)
def import_split(csv_file: str, single_col_file: str, outfile: str,
                 note: str, gtf = '', orif = '', inf = '') -> None:
    # read csv lines
    file3c = open(csv_file)
    triples = file3c.read().splitlines()
    file3c.close()
    
    # read single column file lines
    file1c = open(single_col_file)
    singles = file1c.read().splitlines()
    file1c.close()

    # create the new split file as csv two columns
    with open(os.path.join(outfile), 'w') as out:
        i = 0

        for entry in triples: # oriname.ext, gtname.ext, te/tr/va
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            note_old = entry.split(csv_sep)[2]
            ori_name, ori_ext = os.path.splitext(os.path.basename(ori_path))
            gt_name, gt_ext = os.path.splitext(os.path.basename(gt_path))

            ori_identifier = get_variable_filename(ori_name, orif)
            gt_identifier = get_variable_filename(gt_name, gtf)

            for line in singles: # imgname
                line_name, line_ext = os.path.splitext(line)
                in_identifier = get_variable_filename(line_name, inf)

                if ori_identifier == in_identifier or gt_identifier == in_identifier:
                    note_old = note
                    i += 1
                    print(f'Match found: {ori_identifier}\|{gt_identifier} - {in_identifier}')
                    break # match found
                
            out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note_old}\n")
        
        print(f'''Converted {i}/{len(singles)} lines.\n
        Source file: {single_col_file}\n
        Target file: {outfile}''')

# update the notes in csv by adding a split from a partial file with the same format
def update_split(csv_file: str, partial_file: str, outfile: str, newnote: str) -> None:
    # read csv lines
    file3c = open(csv_file)
    triples = file3c.read().splitlines()
    file3c.close()
    
    # read partial file lines
    file1c = open(partial_file)
    partials = file1c.read().splitlines()
    file1c.close()

    # create the new split file as csv two columns
    with open(os.path.join(outfile), 'w') as out:
        i = 0

        for entry in triples: # oriname.ext, gtname.ext
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            note_old = entry.split(csv_sep)[2]

            skintone = ''
            if len(entry.split(csv_sep)) == 4:
                skintone = csv_sep + entry.split(csv_sep)[3]


            for line in partials: # imgname
                ori_path_part = line.split(csv_sep)[0]

                if ori_path == ori_path_part:
                    note_old = newnote
                    i += 1
                    print(f'Match found: {ori_path}')
                    break # match found
                
            out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note_old}{skintone}\n")
        
        print(f'''Updated {i}/{len(partials)} lines.\n
        Source file: {partial_file}\n
        Target file: {outfile}''')

# import dataset and generate metadata
def import_dataset(import_json: str) -> None:
    if os.path.exists(import_json):
        with open(import_json, 'r') as stream:
            data = json.load(stream)

            # load JSON values
            gt = data['gt']
            ori = data['ori']
            root = data['root']
            note = data['note']
            gt_format = data['gtf']
            ori_format = data['orif']
            gt_ext = data['gtext']
            ori_ext = data['oriext']
            ori_process = data['oriprocess']
            ori_process_out = data['oriprocessout']
            gt_process = data['gtprocess']
            gt_process_out = data['gtprocessout']
            
            # check if processing is required
            if ori_process:
                ori = process_images(ori, ori_process, ori_process_out,
                                     ori_format, ori_ext)
                # update the file extension in the images are being converted
                if 'png' in ori_process:
                    ori_ext = 'png'
            
            if gt_process:
                gt = process_images(gt, gt_process, gt_process_out,
                                     gt_format, gt_ext)
                if 'png' in gt_process:
                    gt_ext = 'png'
            
            # Non-Defined as default note
            if not note:
                note = 'nd'
            
            # analyze the dataset and create the csv files
            analyze_dataset(gt, ori, root,
                            note, gt_format, ori_format,
                            gt_ext, ori_ext)
    else:
        print("JSON import file does not exist!")

def get_timestamp() -> str:
    return time.strftime("%Y%m%d-%H%M%S")

# from schmugge custom config (.config.SkinImManager) to a list of dict structure
def read_schmugge(skin_im_manager_path: str, images_dir: str) -> list: # also prepare the csv
    sch = []
    
    # images with gt errors, aa69 is also duplicated in the config file
    blacklist = ['aa50.gt.d3.pgm', 'aa69.gt.d3.pgm', 'dd71.gt.d3.pgm', 'hh54.gt.d3.pgm']

    with open(skin_im_manager_path) as f:
        start = 0
        i = 0
        tmp = {}
        for line in f:
            blacklisted = False

            if start < 2: # skip first 2 lines
                start += 1
                continue
            
            #print(f'{line}\t{i}') # debug
            if line: # line not empty
                line = line.rstrip() # remove End Of Line (\n)

                if i == 2: # skin tone type
                    skin_type = int(line)
                    if skin_type == 0:
                        tmp['skintone'] = 'light'
                    elif skin_type == 1:
                        tmp['skintone'] = 'medium'
                    elif skin_type == 2:
                        tmp['skintone'] = 'dark'
                    else:
                        tmp['skintone'] = 'nd'
                elif i == 3: # db type
                    tmp['db'] = line
                elif i == 8: # ori
                    tmp['ori'] = os.path.join(images_dir, line)
                elif i == 9: # gt
                    tmp['gt'] = os.path.join(images_dir, line)
                    if line in blacklist:
                        blacklisted = True
                

                # update image counter
                i += 1
                if i == 10: # 10 lines read, prepare for next image data
                    if not blacklisted:
                        sch.append(tmp)
                    tmp = {}
                    i = 0
    
    print(f'Schmugge custom config read correctly, found {len(sch)} images')

    return sch

# from schmugge list of dicts structure to csv file and processed images
def process_schmugge(sch: list, outfile: str, train = 70, test = 15, val = 15, ori_out_dir = 'new_ori', gt_out_dir = 'new_gt'):
    # prepare new ori and gt dirs
    os.makedirs(ori_out_dir, exist_ok=True)
    os.makedirs(gt_out_dir, exist_ok=True)

    with open(outfile, 'w') as out:
        shuffle(sch) # randomize

        # 70% train, 15% val, 15% test
        train_files, test_files = get_training_and_testing_sets(sch)
        test_files, val_files = get_training_and_testing_sets(test_files, split=.5)

        for entry in sch:
            db = int(entry['db'])
            ori_path = entry['ori']
            gt_path = entry['gt']
            

            ori_basename = os.path.basename(ori_path)
            gt_basename = os.path.basename(gt_path)
            ori_filename, ori_e = os.path.splitext(ori_basename)
            gt_filename, gt_e = os.path.splitext(gt_basename)

            # process images
            # load images
            ori_im = cv2.imread(ori_path)
            gt_im = cv2.imread(gt_path)
            # png
            ori_out = os.path.join(ori_out_dir, ori_filename + '.png')
            gt_out = os.path.join(gt_out_dir, gt_filename + '.png')
            # binarize gt: whatever isn't background, is skin
            if db == 4 or db == 3: # Uchile/UW: white background
                b = 255
                g = 255
                r = 255
                lower_val = (b, g, r)
                upper_val = lower_val
                # Threshold the image to get only selected colors
                mask = cv2.inRange(gt_im, lower_val, upper_val)
                #cv2_imshow(mask) #debug
                # what isn't bg is white
                sk = cv2.bitwise_not(mask)
                gt_im = sk
            else: # background = 180,180,180
                b = 180
                g = 180
                r = 180
                lower_val = (b, g, r)
                upper_val = lower_val
                # Threshold the image to get only selected colors
                mask = cv2.inRange(gt_im, lower_val, upper_val)
                #cv2_imshow(mask) #debug
                # what isn't bg is white
                sk = cv2.bitwise_not(mask)
                gt_im = sk
            # save processing 
            cv2.imwrite(ori_out, ori_im)
            cv2.imwrite(gt_out, gt_im)

            skintone = entry['skintone']
            note = 'te'
            if entry in train_files:
                note = 'tr'
            elif entry in val_files:
                note = 'va'
            
            out.write(f"{ori_out}{csv_sep}{gt_out}{csv_sep}{note}{csv_sep}{skintone}\n")

# write all csv line note attributes as the given argument
def csv_full_test(csv_file: str, note = 'nd'):
    # read the images CSV
    file = open(csv_file)
    file3c = file.read().splitlines()
    file.close()

    # rewrite csv file
    with open(csv_file, 'w') as out:
        for entry in file3c:
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            #note = 'nd'

            # check if there is also a 4th parameter in the line (Schmugge skintones)
            skintone = ''
            if len(entry.split(csv_sep)) == 4:
                skintone = csv_sep + entry.split(csv_sep)[3]

            out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note}{skintone}\n")

# write all csv line note attributes as the given argument
def csv_count_test(csv_file: str, count: int, note = 'nd'):
    # read the images CSV
    file = open(csv_file)
    file3c = file.read().splitlines()
    file.close()

    # rewrite csv file
    i = 0
    with open(csv_file, 'w') as out:
        for entry in file3c:
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            #note = 'nd'

            if i < count:
                note = 'te'
                i += 1
            else:
                note = 'tr'

            # check if there is also a 4th parameter in the line (Schmugge skintones)
            skintone = ''
            if len(entry.split(csv_sep)) == 4:
                skintone = csv_sep + entry.split(csv_sep)[3]

            out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note}{skintone}\n")

# all the items with note 'te' become 'tr', all the items with note != 'te', become 'te'
def csv_not_test(csv_file: str):
    # read the images CSV
    file = open(csv_file)
    file3c = file.read().splitlines()
    file.close()

    # rewrite csv file
    with open(csv_file, 'w') as out:
        for entry in file3c:
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            nt = entry.split(csv_sep)[2]

            if nt == 'te':
                note = 'tr'
            else:
                note = 'te'

            # check if there is also a 4th parameter in the line (Schmugge skintones)
            skintone = ''
            if len(entry.split(csv_sep)) == 4:
                skintone = csv_sep + entry.split(csv_sep)[3]

            out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note}{skintone}\n")

# Updates the csv file by modyfing the notes of lines of the given skintone
# csv must have 4 cols!
# skintone may be: 'dark', 'light', 'medium'
def csv_skintone_filter(csv_file: str, skintone: str, mode = 'train', val_percent = .15, test_percent = .15):
    # read the images CSV
    file = open(csv_file)
    file3c = file.read().splitlines()
    file.close()

    # randomize
    shuffle(file3c)

    # calculate splits length
    totalsk = csv_skintone_count(csv_file, skintone) # total items to train/val/test on
    totalva = round(totalsk * val_percent)
    totalte = round(totalsk * test_percent)
    #totaltr = totalsk - totalva
    jva = 0
    jte = 0
    #jtr = 0


    # rewrite csv file
    with open(csv_file, 'w') as out:
        for entry in file3c:
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]

            skint = entry.split(csv_sep)[3]

            if skint != skintone: # should not be filtered
                note = 'nd'
                
                out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note}{csv_sep}{skint}\n")
            else: # should be in the filter
                if mode == 'train': # if it is a training filter
                    if jva < totalva: # there are still places left to be in validation set
                        note = 'va'
                        jva += 1
                    elif jte < totalte: # there are still places left to be in test set
                        note = 'te'
                        jte += 1
                    else: # no more validation places to sit in, go in train set
                        note = 'tr'
                else: # if it is a testing filter, just place them all in test set
                    note = 'te'
                
                out.write(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note}{csv_sep}{skintone}\n")

# Prints the total amount of items of the given skintone
def csv_skintone_count(csv_file: str, skintone: str):
    # read the images CSV
    file = open(csv_file)
    file3c = file.read().splitlines()
    file.close()

    j = 0
    # read csv file
    with open(csv_file, 'r') as out:
        for entry in file3c:
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            note = entry.split(csv_sep)[2]
            skint = entry.split(csv_sep)[3]

            if skint == skintone:
                j += 1
                print(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{note}{csv_sep}{skint}")
    
    print(f"Found {j} items of type {skintone}")
    return j

# Prints the total amount of items of the given mode('train'(includes validation), 'test')
def csv_note_count(csv_file: str, mode: str):
    # read the images CSV
    file = open(csv_file)
    file3c = file.read().splitlines()
    file.close()

    j = 0
    # read csv file
    with open(csv_file, 'r') as out:
        for entry in file3c:
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            nt = entry.split(csv_sep)[2]
            skint = entry.split(csv_sep)[3]

            notes = []
            if mode == 'train':
                notes.append("tr")
                notes.append("va")
            else:
                notes.append("te")

            if nt in notes:
                j += 1
                print(f"{ori_path}{csv_sep}{gt_path}{csv_sep}{nt}{csv_sep}{skint}")
    
    print(f"Found {j} items of type {mode}")

In [ ]:
# assumes the Schmugge dataset data.csv file is in dataset/Schmugge folder
# mode can either be 'train' or 'test'
def gen_sch_by_skintone(skintone: str, mode: str):
    sch_csv = 'dataset/Schmugge/data.csv'

    # re-import Schmugge
    schm = read_schmugge('dataset/Schmugge/data/.config.SkinImManager', 'dataset/Schmugge/data/data')
    process_schmugge(schm, sch_csv, ori_out_dir='dataset/Schmugge/newdata/ori', gt_out_dir='dataset/Schmugge/newdata/gt')

    csv_skintone_filter(sch_csv, skintone, mode = mode)
    csv_skintone_count(sch_csv, skintone)
    #csv_note_count(filter_by_skintone_csv, filter_mode)

# gen_sch_by_skintone('dark', 'train')
# gen_sch_by_skintone('light', 'train')
# gen_sch_by_skintone('medium', 'test')

In [ ]:
# generate datasets metadata

# simple datasets
import_dataset("dataset/import_ecu.json")
#import_dataset("dataset/import_uchile.json")

# hgr is composed of 3 sub datasets
import_dataset("dataset/import_hgr1.json")
import_dataset("dataset/import_hgr2a.json")
import_dataset("dataset/import_hgr2b.json")

# pratheepan is composed of 2 sub datasets
#import_dataset("dataset/import_pratheepanface.json")
#import_dataset("dataset/import_pratheepanfamily.json")

# abd has a native train/test splits
#import_dataset("dataset/import_abd_te.json")
#import_dataset("dataset/import_abd_tr.json")

# vdm dataset is composed of 5 sub datasets with train/test splits
#import_dataset("dataset/import_ami_te.json")
#import_dataset("dataset/import_ami_tr.json")
#import_dataset("dataset/import_ed_te.json")
#import_dataset("dataset/import_ed_tr.json")
#import_dataset("dataset/import_liris_te.json")
#import_dataset("dataset/import_liris_tr.json")
#import_dataset("dataset/import_ssg_te.json")
#import_dataset("dataset/import_ssg_tr.json")
#import_dataset("dataset/import_ut_te.json")
#import_dataset("dataset/import_ut_tr.json")

# schmugge dataset has really different filename formats but has a custom config file included
schm = read_schmugge('dataset/Schmugge/data/.config.SkinImManager', 'dataset/Schmugge/data/data')
process_schmugge(schm, 'dataset/Schmugge/data.csv', ori_out_dir='dataset/Schmugge/newdata/ori', gt_out_dir='dataset/Schmugge/newdata/gt')

Import the train,validation,test sets txts used in the Skinny paper

In [ ]:
# import ECU splits from the Skinny paper

# unzip archive
!unzip -j drive/MyDrive/datasets/sets/ECU_Skinny.zip -d dataset/ECU/

# import splits
import_split('dataset/ECU/data.csv', 'dataset/ECU/train.txt',
             'dataset/ECU/data.csv', 'tr')
import_split('dataset/ECU/data.csv', 'dataset/ECU/test.txt',
            'dataset/ECU/data.csv', 'te')
import_split('dataset/ECU/data.csv', 'dataset/ECU/val.txt',
            'dataset/ECU/data.csv', 'va')



# Import my HGR and Schmugge splits
sch_from = 'drive/MyDrive/training/skinny/checkpoint-20210505-225202/schmugge_datacsv_model.csv' # sch
hgr_from = 'drive/MyDrive/training/skinny/checkpoint-20210512-220723/HGR_data.csv' # hgr
sch_to = 'dataset/Schmugge/data.csv'
hgr_to = 'dataset/HGR_small/data.csv'


!rm $sch_to
!rm $hgr_to

!cp $sch_from $sch_to
!cp $hgr_from $hgr_to

PYTHON Imports

In [ ]:
import os
import time
from abc import abstractmethod
from xml.etree import ElementTree as ET
from typing import Callable, Any

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

import numpy as np

from google.colab.patches import cv2_imshow

import cv2
import json

Define model

In [ ]:
def inception_module(prev_layer, filters: int, activation=layers.LeakyReLU):
    filters = filters // 4
    conv_1 = layers.Conv2D(filters, (1, 1), padding='same')(prev_layer)
    conv_1 = activation()(conv_1)
    conv_3 = layers.Conv2D(filters, (1, 1), padding='same')(prev_layer)
    conv_3 = layers.Conv2D(filters, (3, 3), padding='same')(conv_3)
    conv_3 = activation()(conv_3)
    conv_5 = layers.Conv2D(filters, (1, 1), padding='same')(prev_layer)
    conv_5 = layers.Conv2D(filters, (5, 5), padding='same')(conv_5)
    conv_5 = activation()(conv_5)
    max_pool = layers.MaxPool2D(padding='same', strides=(1, 1))(prev_layer)
    max_pool = layers.Conv2D(filters, (1, 1), padding='same')(max_pool)
    max_pool = activation()(max_pool)
    return tf.concat([conv_1, conv_3, conv_5, max_pool], axis=-1)


def dense_block(prev_layer, filters: int, kernel_size: int or tuple, activation=layers.LeakyReLU):
    dense_1 = layers.Conv2D(filters // 2, kernel_size, padding='same')(prev_layer)
    dense_1 = layers.BatchNormalization()(dense_1)
    dense_1 = activation()(dense_1)
    dense_2 = layers.Conv2D(filters // 4, kernel_size, padding='same')(dense_1)
    dense_2 = layers.BatchNormalization()(dense_2)
    dense_2 = activation()(dense_2)
    dense_3 = layers.Conv2D(filters // 8, kernel_size, padding='same')(dense_2)
    dense_3 = layers.BatchNormalization()(dense_3)
    dense_3 = activation()(dense_3)
    return tf.concat([dense_1, dense_2, dense_3, prev_layer], axis=-1)


def get_filters_count(level: int, initial_filters: int) -> int:
    return 2**(level-1)*initial_filters


class Model:
    def __init__(self, levels: int = None, initial_filters: int = None,
                 image_channels: int = None, log_dir: str = 'logs',
                 load_checkpoint: bool = False, checkpoint_extension: str = 'ckpt',
                 model_name: str = None) -> None:
        self.levels = levels
        self.initial_filters = initial_filters
        self.image_channels = image_channels
        self.keras_model = None
        self.log_dir = log_dir
        self.checkpoint_extension = checkpoint_extension
        if model_name is not None:
            self.name = model_name
        self.load_checkpoint = load_checkpoint
        if not load_checkpoint and os.path.isdir(self.get_logdir()):
            self.name = f"{self.name}_{get_timestamp()}"

    def get_model(self) -> keras.Model:
        path = os.path.join(self.log_dir, self.name, 'checkpoint', f'saved_model.{self.checkpoint_extension}')
        
        if self.load_checkpoint and self.checkpoint_extension != 'ckpt': # load full model
            self.keras_model = load_model(path, compile=False)
            return self.keras_model

        self.keras_model = self.create_model()
        self.__change_model_name()
        self.__plot_model(self.keras_model)

        if self.load_checkpoint: # load weights
            try:
                self.keras_model.load_weights(path)
            except Exception as e:
                raise e
        return self.keras_model

    def __plot_model(self, model: keras.Model) -> None:
        plot_model(model, to_file=os.path.join(self.log_dir, self.name, 'model.png'), show_shapes=True)

    def __change_model_name(self) -> None:
        if self.name is not None and self.keras_model is not None:
            self.keras_model._name = self.name

    def get_logdir(self) -> str:
        return os.path.join(self.log_dir, self.name)

    @abstractmethod
    def create_model(self) -> keras.Model:
        pass


class Skinny(Model):
    name = "Skinny"

    def create_model(self) -> keras.Model:
        self.levels += 1
        kernel_size = (3, 3)
        layers_list = [None for _ in range(self.levels)]
        layers_list[0] = layers.Input(shape=(None, None, self.image_channels), name='feature')
        activation = layers.LeakyReLU

        for i in range(1, self.levels):
            filters = get_filters_count(i, self.initial_filters)
            prev = i-1
            if i != 1:
                layers_list[i] = layers.MaxPool2D()(layers_list[prev])
                prev = i

            layers_list[i] = layers.Conv2D(filters, kernel_size, padding='same')(layers_list[prev])
            layers_list[i] = layers.BatchNormalization()(layers_list[i])
            layers_list[i] = activation()(layers_list[i])
            layers_list[i] = inception_module(layers_list[i], filters, activation)

        for i in range(self.levels-2, 0, -1):
            filters = get_filters_count(i, self.initial_filters)
            layers_list[i+1] = layers.UpSampling2D()(layers_list[i+1])
            layers_list[i+1] = layers.Conv2D(filters, kernel_size, padding='same')(layers_list[i+1])
            layers_list[i+1] = layers.BatchNormalization()(layers_list[i+1])
            layers_list[i+1] = activation()(layers_list[i+1])

            layers_list[i] = dense_block(layers_list[i], filters, kernel_size, activation)

            layers_list[i] = tf.concat([layers_list[i+1], layers_list[i]], axis=-1)
            layers_list[i] = inception_module(layers_list[i], filters, activation)

        layers_list[1] = layers.Conv2D(self.initial_filters, kernel_size, padding='same')(layers_list[1])
        layers_list[1] = activation()(layers_list[1])
        layers_list[1] = layers.Conv2D(self.initial_filters//2, kernel_size, padding='same')(layers_list[1])
        layers_list[1] = activation()(layers_list[1])
        layers_list[1] = layers.Conv2D(1, kernel_size, padding='same',
                                       activation='sigmoid', name='label')(layers_list[1])

        model = keras.Model(inputs=[layers_list[0]], outputs=[layers_list[1]], name=self.name)
        return model

Define PreProcessor

In [ ]:
class Preprocessor:
    def __init__(self):
        self.operations = [] # Preprocessor has no operations by default
    
    # adds a downscale operation to the preprocessing operations list, returns self
    def downscale(self, max_pixel_count):
        # operation that downscales the images composed of more pixels
        # than max_pixel_count preserving the aspect ratio
        def downscale_operation(data):
            for k, v in data.items():
                tensor_shape = tf.cast(tf.shape(v), tf.float32)
                coefficient = max_pixel_count / (tensor_shape[0] * tensor_shape[1])
                coefficient = tf.math.sqrt(coefficient)
                data[k] = tf.cond(coefficient >= 1.0, lambda: v,
                                  lambda: tf.image.resize(v, [tf.cast(tensor_shape[0] * coefficient, tf.uint16),
                                                              tf.cast(tensor_shape[1] * coefficient, tf.uint16)]))
            return data

        self.operations.append(downscale_operation)
        return self
    
    # adds a cast operation to the preprocessing operations list, returns self
    def cast(self, dtype):
        # operation that casts the images data into the given dtype
        def cast_operation(data):
            for k, v in data.items():
                data[k] = tf.cast(v, dtype)
            return data

        self.operations.append(cast_operation)
        return self
    
    # adds a normalize operation to the preprocessing operations list, returns self
    def normalize(self):
        # operation that transforms the images data from uint8(0-255 limited) into floats(0-1 limited)
        def normalize_operation(data):
            for k, v in data.items():
                data[k] = v / 255.0
            return data

        self.operations.append(normalize_operation)
        return self
    
    # adds a padding operation to the preprocessing operations list, returns self
    def pad(self, network_levels):
        number_multiple = 2**(network_levels-1)
        # operation that adds padding to the down and the right of the images
        def padding_operation(data):
            for k, v in data.items():
                tensor_shape = tf.shape(v)
                data[k] = tf.pad(v, [[0, number_multiple - tensor_shape[0] % number_multiple],
                                     [0,  number_multiple - tensor_shape[1] % number_multiple],
                                     [0, 0]])
            return data

        self.operations.append(padding_operation)
        return self
    
    # executes all the operation functions defined in the Preprocessor instance
    # on the given Dataset object and returns the transformed Dataset object
    def add_to_graph(self, dataset) -> tf.data.Dataset:
        for operation in self.operations:
            dataset = dataset.map(operation) # map will execute one function on every element of the Dataset
        return dataset

Define DataLoader

In [ ]:
class DataLoader(object):
    buffer_size = tf.data.experimental.AUTOTUNE

    def __init__(self, dataset_dir: str, batch_size: int, preprocessor: Preprocessor = None):
        self.dataset_dir = dataset_dir
        self.batch_size = batch_size
        self.val_dataset = None
        self.train_dataset = None
        self.test_dataset = None
        self.preprocessor = Preprocessor() if preprocessor is None else preprocessor

    @property
    def train_dataset(self):
        return self.__batch_and_prefetch(self.__train_dataset)

    @train_dataset.setter
    def train_dataset(self, dataset: tf.data.Dataset):
        self.__train_dataset = dataset

    @property
    def test_dataset(self):
        return self.__batch_and_prefetch(self.__test_dataset)

    @test_dataset.setter
    def test_dataset(self, dataset: tf.data.Dataset):
        self.__test_dataset = dataset

    @property
    def val_dataset(self):
        return self.__batch_and_prefetch(self.__val_dataset)

    @val_dataset.setter
    def val_dataset(self, dataset: tf.data.Dataset):
        self.__val_dataset = dataset

    @property
    def preprocessor(self) -> Preprocessor:
        return self.__preprocessor

    @preprocessor.setter
    def preprocessor(self, preprocessor: Preprocessor):
        self.__preprocessor = preprocessor
        self.__reinstantiate()

    def __reinstantiate(self):
        self.train_dataset = self.__create_dataset_pipeline('tr')
        self.val_dataset = self.__create_dataset_pipeline('va', shuffle=False)
        self.test_dataset = self.__create_dataset_pipeline('te', shuffle=False)

    def __batch_and_prefetch(self, dataset: tf.data.Dataset) -> tf.data.Dataset:
        return dataset.\
            padded_batch(self.batch_size, padded_shapes=({'feature': [None, None, 3]}, {'label': [None, None, 1]})).\
            prefetch(buffer_size=self.buffer_size)
    
    def __get_subset_paths(self, subset: str) -> list:
        # read the images CSV (ori_image_filename.ext, gt_image_filename.ext)
        file = open(os.path.join(self.dataset_dir, 'data.csv'))
        file3c = file.read().splitlines()
        file.close()

        files = []

        for entry in file3c:
            ori_path = entry.split(csv_sep)[0]
            gt_path = entry.split(csv_sep)[1]
            note = entry.split(csv_sep)[2]
            
            if note == subset:
                files.append((ori_path, gt_path))
        
        if len(files) == 0:
            print(f'''No files found for subset {subset}!\n
            using the whole dataset instead''')
            for entry in file3c:
                ori_path = entry.split(csv_sep)[0]
                gt_path = entry.split(csv_sep)[1]
                
                files.append((ori_path, gt_path))
        else:
            print(f'Found {subset} split of {len(files)} files')
        
        return files

    def __create_dataset_pipeline(self, subset: str, shuffle: bool = True) -> tf.data.Dataset:
        def process_example_paths(example):
            # TODO bmp should raise errors, tensorflow doesn't like them
            return {'feature': tf.io.decode_image(tf.io.read_file(example[0]), channels=3, expand_animations = False),
                    'label': tf.io.decode_image(tf.io.read_file(example[1]), channels=1,  expand_animations = False)}

        def convert_to_in_out_dicts(example):
            output_dict = {'label': example.pop('label')}
            return example, output_dict

        dataset = self.__get_subset_paths(subset)
        dataset = tf.data.Dataset.from_tensor_slices(dataset)
        dataset = dataset.map(process_example_paths)
        dataset = self.preprocessor.add_to_graph(dataset)
        dataset = dataset.map(convert_to_in_out_dicts).cache()
        if shuffle:
            dataset = dataset.shuffle(2000, reshuffle_each_iteration=True)
        return dataset

In [ ]:
# custom loader used for predictions

def my_batch(dataset: tf.data.Dataset, batch_size) -> tf.data.Dataset:
    return dataset.\
        padded_batch(batch_size, padded_shapes=({'feature': [None, None, 3]})).\
        prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

def my_file(path: str) -> list:
    files = [path]
    return files

def my_loader(path, preprocessor) -> tf.data.Dataset:
    def my_process(example):
        return {'feature': tf.io.decode_image(tf.io.read_file(example), channels=3, expand_animations = False)}
    # def my_inout(example):
    #     output_dict = {'label': example.pop('label')}
    #     return example, output_dict

    #dataset = [(path),]
    dataset = my_file(path)
    dataset = tf.data.Dataset.from_tensor_slices(dataset)
    dataset = dataset.map(my_process)

    t_start = time.time()
    # preprocessing
    dataset = preprocessor.add_to_graph(dataset)
    t_elapsed = time.time() - t_start

    #dataset = dataset.map(my_inout).cache()
    dataset = dataset.cache()
    return dataset, t_elapsed

def single_predict(model, im_path: str, out_path: str, preprocessor):
    os.makedirs(os.path.dirname(out_path), exist_ok = True)

    # get tf Dataset structure containing the image to predict and elapsed preprocessing time
    tf_ds, t_elapsed_pre = my_loader(im_path, preprocessor)

    # predict the image
    for entry in tf_ds:
        # convert to tensor to prevent memory leak https://stackoverflow.com/a/64765018
        tensor = tf.convert_to_tensor(entry['feature'], dtype=tf.float32)
        tensor = tf.expand_dims(tensor, axis=0) # add a dimension

        #model = my_load_model(model_path)
        model.get_model()

        # get time before prediction
        t_start = time.time()

        pred = model.keras_model.predict(tensor) # predict from feature image (X)
        # post-processing
        pred = pred[0]*255 # reshape and de-preprocess

        # prediction + postprocessing
        t_elapsed = time.time() - t_start

        # preprocessing + prediction + postprocessing elapsed time
        t_elapsed_full = t_elapsed_pre + t_elapsed

        # save to a file
        cv2.imwrite(out_path, pred)

        print(t_elapsed_pre)
        print(t_elapsed)
        print(t_elapsed_full)

Define Trainer

In [ ]:
class Trainer:

    def __init__(self, data_loader: DataLoader, model: Model,
                 log_dir: str = './logs', evaluate_test_data=False):
        self.data_loader = data_loader
        self.model = model
        self.metrics = []
        self.losses = []
        self.callbacks = []
        self.log_dir = os.path.join(log_dir, model.name)
        self.timelog = None
        self.evaluate_test_data = evaluate_test_data

    @property
    def model(self) -> Model:
        return self.__model

    @model.setter
    def model(self, model: Model):
        self.__model = model

    @property
    def data_loader(self) -> DataLoader:
        return self.__data_loader

    @data_loader.setter
    def data_loader(self, data_loader: DataLoader):
        self.__data_loader = data_loader

    def add_metrics(self, metrics):
        if type(metrics) is not list:
            metrics = [metrics]
        for metric in metrics:
            self.metrics.append(metric)

    def add_losses(self, losses) -> None:
        if type(losses) is not list:
            losses = [losses]
        for loss in losses:
            self.losses.append(loss)

    def add_callbacks(self, callbacks) -> None:
        if type(callbacks) is not list:
            callbacks = [callbacks]
        for callback in callbacks:
            self.callbacks.append(callback)

    def combined_loss(self):
        def loss(y_true, y_pred):
            result = None
            for i, v in enumerate(self.losses):
                if i == 0:
                    result = v(y_true, y_pred)
                else:
                    result += v(y_true, y_pred)
            return result
        return loss

    def __log_evaluation_metrics(self, metrics: dict):
        root = ET.Element('metrics')
        tree = ET.ElementTree(root)
        for name, value in metrics.items():
            metric_element = ET.SubElement(root, name)
            metric_element.text = str(value)
        tree.write(open(os.path.join(self.model.get_logdir(), 'test_metrics.xml'), 'w'), encoding='unicode')

    def train(self, epochs, optimizer, initial_epoch=0, verbose=1):
        assert self.model is not None, "Model hasn't been set for the trainer."
        assert self.data_loader is not None, "DataLoader hasn't been set for the trainer."
        os.makedirs(self.model.get_logdir(), exist_ok=True)
        model = self.model.get_model()
        model.compile(optimizer=optimizer, loss=self.combined_loss(), metrics=self.metrics)

        model.fit(self.data_loader.train_dataset, validation_data=self.data_loader.val_dataset,
                  epochs=epochs, verbose=verbose, initial_epoch=initial_epoch,
                  callbacks=self.callbacks, shuffle=True)
        if self.evaluate_test_data:
            evaluation_metrics = model.evaluate(self.data_loader.test_dataset, verbose=1)
            evaluation_metrics = dict(zip(model.metrics_names, evaluation_metrics))
            self.__log_evaluation_metrics(evaluation_metrics)

Define WorkScheduler

In [ ]:
class WorkScheduler:
    def __init__(self) -> None:
        self.data = []

    def add_data(self, model: Model,
                 func: Callable[[Model, Any], None],
                 **kwargs) -> None:
        self.data.append((model, func, kwargs))

    def do_work(self) -> None:
        for model, func, kwargs in self.data:
            tf.keras.backend.clear_session()
            tf.compat.v1.reset_default_graph()
            func(model=model, **kwargs)

Define losses

In [ ]:
def dice_loss(y_true, y_pred, epsilon=1e-15):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1, 2, 3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1, 2, 3))
    loss = tf.squeeze(tf.reshape(1 - numerator / denominator, (-1, 1, 1)))
    return loss

Define metrics

In [ ]:
def recall(y_true, y_pred):
    y_pred = K.round(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_score = true_positives / (possible_positives + K.epsilon())
    return recall_score


def precision(y_true, y_pred):
    y_pred = K.round(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_score = true_positives / (predicted_positives + K.epsilon())
    return precision_score


def f1(y_true, y_pred):
    precision_score = precision(y_true, y_pred)
    recall_score = recall(y_true, y_pred)
    return 2 * ((precision_score * recall_score) / (precision_score + recall_score + K.epsilon()))


def iou(y_true, y_pred):
    y_pred = K.round(y_pred)
    intersection = y_true * y_pred
    not_true = 1 - y_true
    union = y_true + (not_true * y_pred)

    return (K.sum(intersection, axis=-1) + K.epsilon()) / (K.sum(union, axis=-1) + K.epsilon())

Define callbacks

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    pass


class ModelCheckpoint(keras.callbacks.ModelCheckpoint, CustomCallback):
    checkpoint_name = 'saved_model.ckpt'

    def set_model(self, model):
        dir_name = os.path.join(self.filepath, 'checkpoint')
        os.makedirs(dir_name, exist_ok=True)
        timestr = get_timestamp()
        self.filepath = os.path.join(F"/content/drive/MyDrive/training/skinny/checkpoint-" + timestr, self.checkpoint_name )
        #self.filepath = os.path.join(F"/content/drive/MyDrive/training/skinny/dark/checkpoint-" + timestr, self.checkpoint_name )
        super().set_model(model)


class ReduceLROnPlateau(keras.callbacks.ReduceLROnPlateau, CustomCallback):
    pass


class ProgbarLogger(keras.callbacks.ProgbarLogger, CustomCallback):
    pass


class EarlyStopping(keras.callbacks.EarlyStopping, CustomCallback):
    pass


class TensorBoard(keras.callbacks.TensorBoard, CustomCallback):
    def set_model(self, model):
        self.log_dir = os.path.join(self.log_dir, 'tensorboard')
        os.makedirs(self.log_dir, exist_ok=True)
        super().set_model(model)

Variables

In [ ]:
# Skinny (full model with inception and dense blocks)
model_name = 'Skinny'

# model settings
levels = 6
initial_filters = 19
image_channels = 3

# train settings
max_epochs = 200
initial_lr = 1e-4
batch_size = 3

# dirs
log_dir = 'logs'
# in the dataset_dir there is a csv file containing all the splits data
#dataset_dir = 'dataset/Schmugge' #'dataset/ECU'

# preprocessing operations
preprocessor = Preprocessor()
preprocessor.cast(dtype=tf.float32).normalize().downscale(max_pixel_count=512**2).pad(network_levels=levels)

Model functions

In [ ]:
def train_function(model: Model, batch_size: int, dataset_dir: str) -> None:
    data_loader = DataLoader(dataset_dir=dataset_dir, batch_size=batch_size, preprocessor=preprocessor)
    trainer = Trainer(data_loader=data_loader, model=model, evaluate_test_data=True)
    trainer.add_losses([K.binary_crossentropy, dice_loss])
    trainer.add_metrics([
        f1,
        iou,
        precision,
        recall
    ])
    trainer.add_callbacks([
        ModelCheckpoint(filepath=model.get_logdir(), verbose=1, save_best_only=True,
                                  monitor='val_f1', mode='max', save_weight_only=False),#save_weights_only=True),
        ReduceLROnPlateau(monitor='val_f1', factor=0.5, verbose=1, mode='max', min_lr=1e-6, patience=5),
        #EarlyStopping(monitor='val_f1', mode='max', patience=10, verbose=1),
        EarlyStopping(monitor='val_f1', mode='max', patience=50, verbose=1), # for dark
        TensorBoard(log_dir=model.get_logdir(), histogram_freq=5)
    ])

    trainer.train(max_epochs, tf.keras.optimizers.Adam(learning_rate=initial_lr), verbose=1)

def test_function(model: Model, dataset_dir: str) -> None:
    data_loader = DataLoader(dataset_dir=dataset_dir, batch_size=1, preprocessor=preprocessor)
    trainer = Trainer(data_loader=data_loader, model=model, evaluate_test_data=True)
    trainer.add_losses([K.binary_crossentropy, dice_loss])
    trainer.add_metrics([
        f1,
        iou,
        precision,
        recall
    ])

    model.get_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    model.keras_model.compile(optimizer=optimizer, loss=trainer.combined_loss(), metrics=trainer.metrics)
    evaluation_metrics = model.keras_model.evaluate(data_loader.test_dataset, verbose=1)
    evaluation_metrics = dict(zip(model.keras_model.metrics_names, evaluation_metrics))
    print(evaluation_metrics)

# save Skinny X preprocessed-images (512**2) to files
# save the images after the preprocessing, before they enter the model
def save_x(model: Model, dataset_dir: str, preprocessor: Preprocessor, out_dir: str = 'x', skip = 0) -> None:
    data_loader = DataLoader(dataset_dir=dataset_dir, batch_size=1, preprocessor=preprocessor)

    os.makedirs(out_dir, exist_ok = True)

    i = skip
    for entry in data_loader.test_dataset:
        i += 1

        # dict: {'feature': data, dtype=float32}
        entry = entry[0]
        # shape=(1, 320, 384, 1) dtype=float32
        entry = entry['feature']
        # reshape(320, 384, 3) and de-preprocess
        entry = entry[0]*255
        # convert to numpy array or cv2.imwrite doesn't work
        entry = np.array(entry)

        # cv2 works with BGR, not RGB
        entry = cv2.cvtColor(entry, cv2.COLOR_RGB2BGR)

        # save to a file
        filename = f"{i}.png"
        cv2.imwrite(os.path.join(out_dir, filename), entry)
        #tf.keras.preprocessing.image.save_img(os.path.join(out_dir, filename), entry)
        # https://stackoverflow.com/a/61041738

# save Skinny Y preprocessed-images (512**2) to files
# save the images after the preprocessing, before they enter the model
def save_y(model: Model, dataset_dir: str, preprocessor: Preprocessor, out_dir: str = 'y', skip = 0) -> None:
    data_loader = DataLoader(dataset_dir=dataset_dir, batch_size=1, preprocessor=preprocessor)

    os.makedirs(out_dir, exist_ok = True)

    i = skip
    for entry in data_loader.test_dataset:
        i += 1

        # dict: {'label': data, dtype=float32}
        entry = entry[1]
        # shape=(1, 320, 384, 1) dtype=float32
        entry = entry['label']
        # reshape(320, 384, 1) and de-preprocess
        entry = entry[0]*255
        # convert to numpy array or cv2.imwrite doesn't work
        entry = np.array(entry)

        # save to a file
        filename = f"{i}.png"
        cv2.imwrite(os.path.join(out_dir, filename), entry)

def predict_function(model: Model, dataset_dir: str, out_dir: str, skip = 0) -> None:
    data_loader = DataLoader(dataset_dir=dataset_dir, batch_size=1, preprocessor=preprocessor)

    model.get_model()

    os.makedirs(out_dir, exist_ok = True)

    i = skip
    for entry in data_loader.test_dataset: # prova a predirre le immagini di test
        i += 1

        # dict: {'feature': data, 'types': float32}
        entry = entry[0]

        # convert to tensor to prevent memory leak https://stackoverflow.com/a/64765018
        tensor = tf.convert_to_tensor(entry['feature'], dtype=tf.float32)
        #pred = model.keras_model.predict(tensor) # predict from feature image (X)
        pred = model.keras_model(tensor) # predict from feature image (X)
        #print(pred)

        pred = pred[0]*255 # reshape and de-preprocess
        #print(pred)
        pred = pred.numpy()

        # save to a file
        filename = f"{i}.png"
        cv2.imwrite(os.path.join(out_dir, filename), pred)

Loading models

In [ ]:
def load_checkpoint(checkpoint_filepath):
    out = None
    ext = os.path.splitext(checkpoint_filepath)[1]

    if ext == '.chkp':
        load_chkpt(checkpoint_filepath)
        out = 'chkp'
    elif ext == '.pb':
        load_pb(checkpoint_filepath)
        out = 'pb'
    else:
        print(f'Unknown model filetype: {ext}')
    
    return out

def load_chkp(chkp_index_filepath):
    # clear checkpoints
    !rm -rf logs/Skinny/checkpoint

    # define from/to loading directories
    from_path = os.path.join(os.path.dirname(chkp_index_filepath), '.')
    to_path = 'logs/Skinny/checkpoint'

    # make default model folder
    !mkdir -p $to_path

    # copy the model files
    !cp -r $from_path $to_path

def load_pb(pb_filepath):
    # clear checkpoints
    !rm -rf logs/Skinny/checkpoint

    # define from/to loading directories
    from_path = os.path.join(os.path.dirname(pb_filepath), '.')
    to_path = 'logs/Skinny/checkpoint/saved_model.pb'

    # make default model folder
    !mkdir -p $to_path

    # copy the model files
    !cp -r $from_path $to_path

def load_schmugge_skintone_split(skintone):
    !rm dataset/Schmugge/data.csv

    if skintone == 'dark':
        !cp drive/MyDrive/training/skinny/checkpoint-20210523-110554/dark2305_1309.csv dataset/Schmugge/data.csv
        print(f'{skintone}(sch) split imported!')
    elif skintone == 'medium':
        !cp drive/MyDrive/training/skinny/checkpoint-20210523-112308/medium2305_1323.csv dataset/Schmugge/data.csv
        print(f'{skintone}(sch) split imported!')
    elif skintone == 'light':
        !cp drive/MyDrive/training/skinny/checkpoint-20210523-122027/light2305_1420.csv dataset/Schmugge/data.csv
        print(f'{skintone}(sch) split imported!')
    else:
        print(f'no split found for (sch) skintone: {skintone}')

In [ ]:
# Add some already existing trained models to a dictionary for ease of access

models = {}

models['ecu'] = 'drive/MyDrive/training/skinny/checkpoint-20210428-155148/saved_model.ckpt/saved_model.pb' # ecu
models['schmugge'] = 'drive/MyDrive/training/skinny/checkpoint-20210505-225202/saved_model.ckpt/saved_model.pb' # sch
models['hgr'] = 'drive/MyDrive/training/skinny/checkpoint-20210512-220723/saved_model.ckpt/saved_model.pb' # hgr

models['dark'] = 'drive/MyDrive/training/skinny/checkpoint-20210523-110554/saved_model.ckpt/saved_model.pb' # dark
models['medium'] = 'drive/MyDrive/training/skinny/checkpoint-20210523-112308/saved_model.ckpt/saved_model.pb' # medium
models['light'] = 'drive/MyDrive/training/skinny/checkpoint-20210523-122027/saved_model.ckpt/saved_model.pb' # light

TRAIN

In [ ]:
dataset_dir = 'dataset/Schmugge'

#mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True, model_name=model_name)
mod = Skinny(levels, initial_filters, image_channels, log_dir)  # creates new model

#self.keras_model = load_model(path, compile=False)
#mod = tf.saved_model.load('models')

scheduler = WorkScheduler()
scheduler.add_data(mod, train_function, batch_size = batch_size, dataset_dir = dataset_dir)
scheduler.do_work()

TEST

In [ ]:
#chkp_ext = 'pb' # pb or chkp
dataset_dir = 'dataset/abd-skin'

mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True,
             model_name=model_name, checkpoint_extension=chkp_ext)

scheduler = WorkScheduler()
scheduler.add_data(mod, test_function, dataset_dir = dataset_dir)
scheduler.do_work()

PREDICT

In [ ]:
def cross_predict(train_db, predict_db, timestr = None, save = False):
    is_ecu = False
    if predict_db == 'ecu':
        predict_db = 'dataset/ECU'
        is_ecu = True
    elif predict_db == 'hgr':
        predict_db = 'dataset/HGR_small'
    elif predict_db == 'schmugge':
        predict_db = 'dataset/Schmugge'
    
    if timestr == None:
        timestr = get_timestamp()

    # set the whole dataset as the testing set
    whole_test = os.path.join(predict_db, 'data.csv') # dataset to process

    if is_ecu:
        csv_count_test(whole_test, 2000, 'te') # limit ecu to 2000 predictions or ram crash, do it 2 times
    else:
        csv_full_test(whole_test, 'te')

    # load model files
    chkp_ext = load_checkpoint(models[train_db])
    mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True,
            model_name=model_name, checkpoint_extension=chkp_ext)

    # predict
    ds_name = os.path.basename(predict_db).lower()
    if ds_name == 'hgr_small':
        ds_name = 'hgr'


    out_dir = os.path.join(timestr, 'skinny', 'cross', f'{train_db}_on_{ds_name}')
    x_dir = os.path.join(out_dir, 'x')
    y_dir = os.path.join(out_dir, 'y')
    pred_dir = os.path.join(out_dir, 'p')

    scheduler = WorkScheduler()
    scheduler.add_data(None, save_x, dataset_dir = predict_db, out_dir = x_dir, preprocessor = preprocessor)
    scheduler.add_data(None, save_y, dataset_dir = predict_db, out_dir = y_dir,  preprocessor = preprocessor)
    scheduler.add_data(mod, predict_function, dataset_dir = predict_db, out_dir = pred_dir)
    scheduler.do_work()

    if is_ecu: # time to do second half
        csv_not_test(whole_test)
        scheduler = WorkScheduler()
        scheduler.add_data(None, save_x, dataset_dir = predict_db, out_dir = x_dir, preprocessor = preprocessor, skip = 2000)
        scheduler.add_data(None, save_y, dataset_dir = predict_db, out_dir = y_dir,  preprocessor = preprocessor, skip = 2000)
        scheduler.add_data(mod, predict_function, dataset_dir = predict_db, out_dir = pred_dir, skip = 2000)
        scheduler.do_work()


    # zip and save predictions
    if save:
        zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_p.zip'
        !zip -r $zip_path $timestr


def base_predict(db_name, timestr = None, save = False):
    if db_name == 'ecu':
        ds = 'dataset/ECU'
    elif db_name == 'hgr':
        ds = 'dataset/HGR_small'
    elif db_name == 'schmugge':
        ds = 'dataset/Schmugge'
    
    if timestr == None:
        timestr = get_timestamp()
    
    # load model files
    chkp_ext = load_checkpoint(models[db_name])
    mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True,
            model_name=model_name, checkpoint_extension=chkp_ext)


    out_dir = os.path.join(timestr, 'skinny', 'base', db_name)
    x_dir = os.path.join(out_dir, 'x')
    y_dir = os.path.join(out_dir, 'y')
    pred_dir = os.path.join(out_dir, 'p')

    scheduler = WorkScheduler()
    scheduler.add_data(None, save_x, dataset_dir = ds, out_dir = x_dir, preprocessor = preprocessor)
    scheduler.add_data(None, save_y, dataset_dir = ds, out_dir = y_dir,  preprocessor = preprocessor)
    scheduler.add_data(mod, predict_function, dataset_dir = ds, out_dir = pred_dir)
    scheduler.do_work()

    # zip and save predictions
    if save:
        zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_p.zip'
        !zip -r $zip_path $timestr


def cross_predict_skintones(train_skintone, predict_skintone, timestr = None, save = False):
    db_dir = 'dataset/Schmugge'

    if timestr == None:
        timestr = get_timestamp()

    # update the csv file to set the prediction set
    gen_sch_by_skintone(predict_skintone, 'test')

    # load model files
    chkp_ext = load_checkpoint(models[train_skintone])
    mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True,
            model_name=model_name, checkpoint_extension=chkp_ext)

    # predict

    out_dir = os.path.join(timestr, 'skinny', 'cross', f'{train_skintone}_on_{predict_skintone}')
    x_dir = os.path.join(out_dir, 'x')
    y_dir = os.path.join(out_dir, 'y')
    pred_dir = os.path.join(out_dir, 'p')

    scheduler = WorkScheduler()
    scheduler.add_data(None, save_x, dataset_dir = db_dir, out_dir = x_dir, preprocessor = preprocessor)
    scheduler.add_data(None, save_y, dataset_dir = db_dir, out_dir = y_dir,  preprocessor = preprocessor)
    scheduler.add_data(mod, predict_function, dataset_dir = db_dir, out_dir = pred_dir)
    scheduler.do_work()

    # zip and save predictions
    if save:
        zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_p.zip'
        !zip -r $zip_path $timestr

def base_predict_skintones(skintone, timestr = None, save = False):
    db_dir = 'dataset/Schmugge'

    if timestr == None:
        timestr = get_timestamp()
    
    # load skintone split
    load_schmugge_skintone_split(skintone)
    
    # load model files
    chkp_ext = load_checkpoint(models[skintone])
    mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True,
            model_name=model_name, checkpoint_extension=chkp_ext)


    out_dir = os.path.join(timestr, 'skinny', 'base', skintone)
    x_dir = os.path.join(out_dir, 'x')
    y_dir = os.path.join(out_dir, 'y')
    pred_dir = os.path.join(out_dir, 'p')

    scheduler = WorkScheduler()
    scheduler.add_data(None, save_x, dataset_dir = db_dir, out_dir = x_dir, preprocessor = preprocessor)
    scheduler.add_data(None, save_y, dataset_dir = db_dir, out_dir = y_dir,  preprocessor = preprocessor)
    scheduler.add_data(mod, predict_function, dataset_dir = db_dir, out_dir = pred_dir)
    scheduler.do_work()

    # zip and save predictions
    if save:
        zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_p.zip'
        !zip -r $zip_path $timestr

In [ ]:
mode = 'skintones' # 'normal' or 'skintones'


timestr = get_timestamp()

if mode == 'normal':
    modls = ['ecu', 'hgr', 'schmugge']

    # base predictions: based on splits defined by me and only predict on self
    # timestr/skinny/base/{ecu,hgr,schmugge}/{p/y/x}
    for ds_name in modls:
        base_predict(ds_name, timestr = timestr, save = False)

    # cross predictions: use a dataset whole as the testing set
    # timestr/skinny/cross/ecu_on_ecu/{p/y/x}
    for ds_train in modls:
        for ds_test in modls:
            cross_predict(ds_train, ds_test, timestr = timestr, save = False)

    # zip and save predictions
    zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_p.zip'
    !zip -r $zip_path $timestr
elif mode == 'skintones':
    modls = ['light', 'medium', 'dark']

    # base predictions: based on splits defined by me and only predict on self
    # timestr/skinny/base/{ecu,hgr,schmugge}/{p/y/x}
    for ds_name in modls:
        base_predict_skintones(ds_name, timestr = timestr, save = False)

    # cross predictions: use a dataset whole as the testing set
    # timestr/skinny/cross/ecu_on_ecu/{p/y/x}
    for ds_train in modls:
        for ds_test in modls:
            cross_predict_skintones(ds_train, ds_test, timestr = timestr, save = False)

    # zip and save predictions
    zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_p.zip'
    !zip -r $zip_path $timestr

In [ ]:
timestr = get_timestamp()
skintones = ['light', 'medium', 'dark']
ds = 'dataset/Schmugge'

#for sk_tr in skintones:
for sk_tr in ['dark']:
    # load training model
    chkp_ext = load_checkpoint(models[sk_tr])

     # predict on train set
    for sk_te in skintones:
        out_dir = f'{sk_tr}_on_{sk_te}'
        x_dir = os.path.join(timestr, out_dir, 'x')
        y_dir = os.path.join(timestr, out_dir, 'y')
        pred_dir = os.path.join(timestr, out_dir, 'p')

        # Prepare test set
        # re-import Schmugge
        schm = read_schmugge('dataset/Schmugge/data/.config.SkinImManager', 'dataset/Schmugge/data/data')
        process_schmugge(schm, 'dataset/Schmugge/data.csv', ori_out_dir='dataset/Schmugge/newdata/ori', gt_out_dir='dataset/Schmugge/newdata/gt')
        filter_by_skintone_type = sk_te # light medium dark nd
        filter_by_skintone_csv = 'dataset/Schmugge/data.csv' # dataset to process
        filter_mode = 'test'
        # filter to create testing set
        csv_skintone_filter(filter_by_skintone_csv, filter_by_skintone_type, mode = filter_mode)
        csv_skintone_count(filter_by_skintone_csv, filter_by_skintone_type)

        # Predict
        mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True,
                model_name=model_name, checkpoint_extension=chkp_ext)

        scheduler = WorkScheduler()
        scheduler.add_data(None, save_x, dataset_dir = ds, out_dir = x_dir, preprocessor = preprocessor)
        scheduler.add_data(None, save_y, dataset_dir = ds, out_dir = y_dir,  preprocessor = preprocessor)
        scheduler.add_data(mod, predict_function, dataset_dir = ds, out_dir = pred_dir)
        scheduler.do_work()

# zip and save predictions
zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_p.zip'
!zip -r $zip_path $timestr

In [ ]:
# Attempts to single predict

# preprocessing operations
preprocessor = Preprocessor()
preprocessor.cast(dtype=tf.float32).normalize().downscale(max_pixel_count=512**2).pad(network_levels=levels)

chkp_ext = load_checkpoint(models['ecu'])
#model_dir = 'logs/Skinny/checkpoint/saved_model.pb'
inim = 'dataset/ECU/origin_images/im00001.jpg'
outim = 'spred/im00001.png'

mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True,
        model_name=model_name, checkpoint_extension=chkp_ext)
single_predict(mod, inim, outim, preprocessor)
#prediction = get_prediction('dataset/org/features/im00043.jpg', preprocessor, my_load_model('logs/Skinny2/checkpoint/saved_model.ckpt'))



SAVE Xs and Ys

In [ ]:
# timestr = time.strftime("%Y%m%d-%H%M%S")
timestr = get_timestamp()
x_dir = os.path.join(timestr, 'x')
y_dir = os.path.join(timestr, 'y')

# all but normalize, TODO right?
pre = Preprocessor()
pre.cast(dtype=tf.float32).downscale(max_pixel_count=512**2).pad(network_levels=levels)

prefull = Preprocessor()
prefull.cast(dtype=tf.float32).normalize().downscale(max_pixel_count=512**2).pad(network_levels=levels)

scheduler = WorkScheduler()
scheduler.add_data(None, save_x, dataset_dir = dataset_dir, out_dir = x_dir, preprocessor = pre)
scheduler.add_data(None, save_y, dataset_dir = dataset_dir, out_dir = y_dir,  preprocessor = prefull)
scheduler.do_work()

In [ ]:
# load files to drive as ZIP or else it will be really slow to reload them into colab

zip_path = 'drive/MyDrive/testing/skinny/' + timestr + '_xy.zip'

# zip
!zip -r $zip_path $timestr

-- OLD

In [ ]:
data_loader = DataLoader(dataset_dir=r'dataset', batch_size=1, preprocessor=preprocessor)
tensor_dataset = create_dataset_pipeline(self, 'images', shuffle = False)

# model.predict(tensor_dataset) # and save them to a folder, maybe see benchmark notebook methods

In [ ]:
# load saved models

!mkdir -p logs/Skinny/checkpoint/saved_model.ckpt
!cp -r drive/MyDrive/training/skinny/checkpoint-/20210411-225022/saved_model.ckpt/. logs/Skinny/checkpoint/saved_model.ckpt

!mkdir -p logs/Skinny2/checkpoint
!cp -r drive/MyDrive/training/skinny/. logs/Skinny2/checkpoint

In [ ]:
# directory
# eg [ckpt]: logs/Skinny/checkpoint/saved_model.ckpt
#            (saved_model.ckpt is neither a file nor a folder, but in the
#             checkpoint folders there are 3 files:
#             checkpoint, saved_model.ckpt.index, saved_model.ckpt.data-00000-of-00001)
# eg [SavedModel]: logs/Skinny/checkpoint/saved_model.ckpt
#                  (saved_model.ckpt is a folder and contains saved_model.pb)
def my_load_model(directory, load_weights: bool = True):
    mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True, model_name=model_name)
    mod.get_model()

    if load_weights:
        path = os.path.join(directory)
        mod.keras_model.load_weights(path)
    #else:
    #    mod = Skinny(levels, initial_filters, image_channels, log_dir, load_checkpoint=True, model_name=model_name)
    
    #mod.get_model()
    return mod

In [ ]:
# path = 'dataset/org/features/im00002.jpg'
def get_prediction(path, preprocessor, model):
    ds = my_loader(path, preprocessor)
    ds = my_batch(ds, 1)
    
    trainer = Trainer(data_loader=ds, model=model, evaluate_test_data=True)
    trainer.add_losses([K.binary_crossentropy, dice_loss])
    trainer.add_metrics([
        f1,
        iou,
        precision,
        recall
    ])

    model.get_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    model.keras_model.compile(optimizer=optimizer, loss=trainer.combined_loss(), metrics=trainer.metrics)

    for string_, int_ in ds:
        img = string_
        break
    
    arr = np.array([img['feature'].numpy()[0]])
    prediction = model.keras_model.predict(arr)
    return prediction[0]*255

#prediction = get_prediction('dataset/org/features/im00003.jpg', preprocessor, my_load_model('logs/Skinny/checkpoint/saved_model.ckpt', load_weights=False))
prediction = get_prediction('dataset/org/features/im00043.jpg', preprocessor, my_load_model('logs/Skinny2/checkpoint/saved_model.ckpt'))
cv2_imshow(prediction)